# <font color='DarkSlateBlue'>Udacity - Intro Data Analysis</font>
## <font color='NavyBlue'>Titanic Data</font>

__ "O Titanic foi pensado para ser o navio mais luxuoso e mais seguro de sua época, gerando lendas que era supostamente "inafundável"." __ 

A viagem que cuminou com o naufrágio do RMS Titanic teve inicio em Southampton/Reino Unido em 10/04/1912, passando por Cherbourg-Octeville/França e por Queenstown/Irlanda. No dia 14 de abril ele colidiu com um iceberg e afundou, na madrugada seguinte, com mais 1500 pessoas a bordo.

Um fato curioso, que pode nos auxiliar na análise de dados, foi que durante a evacuação dos passageiros o capitão Smith dirigiu-se para dois oficiais (Lightoller e Murdoch)  e disse: "coloque as mulheres e crianças e abaixe-os". Os dois interpretaram a ordem de forma diferente, sendo que Ligthtoller entendeu que somente podiam embarcar mulheres e crianças e quando não havia ninguém desse grupo, ele lançava os botes vazios. Murdoch permitia que homens embarcassem depois das mulheres e crianças. Por não saberem o total de pessoas que o bote suportava, vários foram lançados com metade de sua capacidade sem utilização.

Fonte: https://pt.wikipedia.org/wiki/RMS_Titanic

### Questões que irão guiar a análise:

1. Sabendo que houve esse mal entendido na distribuição dos passageiros por botes, a fonte de dados titanic.cvs nos fornece alguma informação que reflete esse mal entendido entre os oficiais?
2. Mulheres e crianças tiveram maior chance de sobreviver?
3. A classe, foi um fator importante para a sobrevivência? 
4. E a pergunta me faço é se eu estivesse lá, mulher, jovem adulta, na 2º classe é provável que teria sobrevivido?

Chega de dúvidas, vamos iniciar a análise dos dados!

## Preparando o ambiente: Importando as bibliotecas e os dados

In [ ]:
#importando as bibliotecas
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import pandas as pd
import numpy as np

In [ ]:
#Machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn import  model_selection, tree, preprocessing, metrics

In [ ]:
filename = 'C:/Nanodegree/Intro_Data_Analys/titanic_data.csv'
titanic_df = pd.read_csv(filename, header=0)

In [ ]:
titanic_df.head()

## Trabalhando os dados

Nesta fase vamos fazer uma faxina nos dados e retirar tudo que possa interferir no resultado da análise e da predição de dados.
Como tudo, você pode considerar valores nulos, colunas com muitos valores nulos, colunas com informações que não pertinentes para o foco do análise, valores muito acima ou abaixo da média e valores zerados.
Os números e a decisão de manter ou não um dado será demonstrado nesta seção.

**Conhecendo melhor as colunas do dataset:**
- Embarked: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton)
- Parch: Número de pais e filhos a bordo
- Pclass: Classe que o passageiro embarcou (1 = 1º classe; 2 = 2º classe; 3 = 3º classe)
- Sex: Gênero feminino (female) ou male (masculino)
- Sibsp: Numero de irmãos e esposa a bordo
- Survived: 0 - Não sobreviveu e 1 = sobreviveu


Pelas questões que foram levantandas vamos assumir que as variáveis que devem ser investigadas mais afundo são:
- Idade
- Sexo
- Acompanhantes: SibSp + Parch
- Classe

In [ ]:
# Verifica  o tipo dos dados criados
titanic_df.dtypes

### Manipulando as variáveis

#### 1º Tipo de manipulação: Valores ausentes

A existência de colunas com dados ausentes será verificada porque estas ausências podem interferir na análise. 
O tratamento do dado ausente levará em consideração o percentual (%) de ausências e a relevância da informação para a conclusão. 

Começaremos este processo identificando quais são as colunas com valores ausentes através da contagem dos valores existentes e divisão pelo total de linhas da coluna.

In [ ]:
((len(titanic_df) - titanic_df.count()) / len(titanic_df)) *100

** Valores ausentes: Age **

20% dos dados da coluna Age estão ausentes. Sabemos que a idade foi um dos critérios de seleção que permitia o passageiro entrar ou não no bote salva vidas. Então, presumindo uma importância, não vou optar por excluir a coluna ou as linhas sem valores. Para este caso, uma boa estratégia será atribuir a média da coluna para em todos ausentes.

In [ ]:
titanic_df.Age.fillna(0).describe()

Atribuindo a média da coluna em todas as células com valores NaN:

In [ ]:
titanic_df.Age = titanic_df.Age.fillna(titanic_df.Age.mean())

**Valores ausentes: Cabin**

In [ ]:
titanic_df.Cabin.describe()

* 77% da coluna é composta por valores nulos. Dos 204 valores existentes, 147 são únicos e os registros mais repetidos 
aparecem apenas 4 vezes. Como top foram listados os valores C23, C25, C27, todos eles são iniciados pela letra C, o que pode 
indicar a existência de um padrão na montagem do campo. Esta possibilidade não será investigada nesta análise, e o campo será 
excluído devido ao alto percentual de nulos.

** Valores ausentes: Embarked ** 

Embarked tem pouquíssimos valores ausentes (0.2%), nesse caso seria cabível até excluir estes registros, mas o porto onde o passageiro embarcou não tem relevância na ótica desta análise. Esta coluna será excluida por completo ao fim desta seção.

In [ ]:
# Embarked tem valores nulos
titanic_df.Embarked.describe()

#### 2º Tipo de manipulação: Dados derivados 

Durante a análise pode ser necessário criar outros dados a partir dos existentes, esse recurso será usado sempre que for identificado um benefício em ver a informação sob outra perspectiva ou acessá-la mais facilmente.

** Dados derivados: Acompanhantes (Family) **

* As variáveis SibSp e Parch tem a mesma raiz de informação (total de acompanhantes), para facilitar a análise, será criada 
uma nova coluna derivada chamada Family que conterá o total de acompanhantes (SibSp + Parch). Estar acompanhado tem alguma importância para a sobrevivência? Ter toda a informação em uma só coluna facilitará a análise.

In [ ]:
# Cria a coluna Family
titanic_df['Family'] = titanic_df['SibSp'] + titanic_df['Parch']

** Dados derivados: Faixa etária (AgeRange) **

A coluna AgeRange será criada para facilitar a análise da faixa etária sobrevivente. Apesar de ter 32 anos, expandirei a pesquisa para faixa etária para que a análise não fique tão pontual. 

In [ ]:
# Descobre a faixa etária baseado na idade do passageiro(Titanic_df.Age)
def age_range(idade):
    """ 
    Retorna a faixa etária para a idade informada.

    Args:
        idade: Valor número que representa a idade.
    Returns:
        Retorna uma string com a faixa etária identificada para idade.
        Domínio de valores = Idoso, Adulto, Jovem Adulto, Adolescente e Crianca.
    """    
    
    if idade >= 65:
        return 'Idoso'
    elif idade >= 33:
        return 'Adulto'
    elif idade >= 18:
        return 'Jovem Adulto'
    elif idade >= 12:
        return 'Adolescente'
    else:
        return 'Crianca'    

In [ ]:
# Chama a função age_range passando como o parametro a coluna Age e atribui o resultado a nova coluna AgeRange
titanic_df['AgeRange']= titanic_df.Age.apply(age_range) 

In [ ]:
titanic_df.AgeRange.describe()

O meu grupo "Jovem Adulto" é o mais frequente do conjunto de dados. Veja a distribuição dos dados:

In [ ]:
# Imprime gráfico com o total de passageiros por faixa etária.
titanic_df.groupby(['AgeRange']).size().plot()

#### 3º Tipo de manipulação: Dados irrelevantes ou inadequados 

- Embarked, Ticket, PassengerId e Name são colunas irrelevantes para o contexto da análise. 
- Saber onde Nome do passageiro, ou seu número de identificação não irá interferir no tipo de análise que será feito.
- Embarked e Ticket são variáveis que podem, em outra análise, serem aprofundadas. Isto não será realizado neste estudo.

O último passo da manipulação é excluir do conjunto de dados as variáveis consideradas irrelevantes ou inadequadas. SibSp e Parch serão excluídas porque o conteúdo delas está replicado na coluna Family.

In [ ]:
# Apaga as colunas
titanic_df.drop(['PassengerId','SibSp', 'Parch', 'Cabin','Embarked', 'Ticket','Name'], axis=1, inplace=True)

titanic_df.head()

## Sumário estatístico
Aplicada aos valores numéricos

- count: Total de elementos não nulos
- mean: Média
- std: Desvio Padrão
- min: Maior valor da amostra
- max: Menor valor da amostra
- 25%: Quartil 1, corresponde a 25% da amostra
- 50%: Quartil 2 ou mediana da amostra
- 75%: Quartil 3, corresponde a % da amostra

In [ ]:
titanic_df.describe()

**Observações:**

* Todas as linhas de todas as colunas estão preenchidas com 891 registros cada.
* A idade dos passageiros está entre 0.42 e 80 anos. 

### Toda hora é hora de trabalhar os dados 

Apesar de ter tratado os dados na seção 'Trabalhando os dados', ao desrever os dados no 'Sumário estatístico', percebi que o conjunto de dados passa a impressão de que os passageiros estavam acompanhados, o menor valor de passagem é 0 e o maior é 512, bem distante da média que é 32.

Vamos investigar e tratar essas variáveis.

* Média de Family

A média de Family é 0.9, dando a impressão de que a maior parte dos passageiros estava acompanhada. 
Confirmando essa informação:

In [ ]:
((len(titanic_df.Family) - titanic_df[titanic_df['Family']>0].count()) / len(titanic_df.Family)) *100

60% dos passageiros estavam desacompanhados. Bem, se a maioria dos passageiros estava desacompanhado, é possível que o total de acompanhantes seja próximo do total de passageiros.  

In [ ]:
# Soma dos registros de Family
titanic_df.Family.sum()

O total da soma de familiares é quase similar ao total de valores da coluna Family. Quando calculado a média, fica a impressão de a maioria dos passageiros esta acompanhado. 

Verificando se algum padrão neste dado:

In [ ]:
# Lista os valores únicos da quantidade de acompanhantes por passageiros
titanic_df.Family.unique()

Já sabemos que a maior parte dos passageiros estava desacompanhada, será que há uma relação diferente desta variável com os mortos e sobreviventes?

Agrupa os dados de family e Survived, com o objetivo de descobrir se ha uma relação entre o número de acompanhantes  e a sobrevivência. 

In [ ]:
acompanhante_sobrevivencia = titanic_df.groupby(['Family','Survived']).size()
acompanhante_sobrevivencia

Visualizando o mapa da distribuição dos sobreviventes por número de acompanhantes:

In [ ]:
sb.heatmap(acompanhante_sobrevivencia.unstack(), annot=True, fmt='g')
plt.xlabel('0 - Mortos , 1 - Sobreviventes')
plt.ylabel('Total Acompanhantes')

Através do gráfico é possível confirmar que a maior parte dos passageiros estava desacompanhando. Se a análise se baseasse apenas nestas duas colunas, sem considerar classe, sexo e o valor da passagem, estar desacompanhado comporia o grupo com menor probabilidade de sobreviver.

#### 4º Tipo de manipulação: Dados discrepantes (outliers)

Tratamos dados ausentes, irrelevantes e derivados, agora encontramos um novo tipo de dado que precisa ser manipulado, porque a discrepância de seu conteúdo pode influenciar/induzir significativamente o resultado final.

** Dados discrepantes: Fare **

Fare tem duas questões que precisa ser avaliada. A primeira é o preço máximo do ticket é 512 sendo que a o valor médio 32. Vou aprofundar na análise desse dados porque estes outliers podem interferir no resultado. A segunda é que o valor mínimo é 0. Pode ser, por exemplo, tickets de crianças. Isto será verificado a seguir.

Questão 1: O maior valor de Fare é realmente discrepante? 

In [ ]:
# Ordena os dados em ordem descente para verificar se existem muitos valores altos como o max=512.
titanic_df.Fare.sort_values(ascending=False).head(30)

O maior valor (512.3292) está muito distante do segundo maior valor (263.0000). Os outros valores são bem aproximados um dos outros. A menos que existam 3 cabines de luxo com uma tarifa exorbitante, o que eu não acredito, há provavelmente uma falha nos dados. Para os passageiros com a passagem igual ao maior valor, vou atribuir o segundo maior valor.  

In [ ]:
# Lista os 10 maiores valores de Fare, mas a lista tem valores repetidos, então filtra
# somente unicos(.unique) e retorna a segunda linha [1], porque a primeira é o Max. 
second_max_fare = titanic_df.Fare.nlargest(10).unique()[1]
second_max_fare

In [ ]:
# Atribui o segundo valor mais alto da coluna(second_max_fare) 
# onde Fare é igual ao valor máximo da coluna
titanic_df.Fare = titanic_df.Fare.apply(lambda x: second_max_fare if x==titanic_df.Fare.max() else x)

In [ ]:
# Confere se o max foi alterado para o segundo valor.
titanic_df.Fare.max()

Questão 2: Os valores zerados são de crianças ou idosos? Eles tem algum fator em comum?

In [ ]:
# Pesquisa resultado onde o valor da passagem é igual a 0.
titanic_df[titanic_df['Fare']==0]

A teoria de que podiam ser ingressos gratuitos por causa da idade não se confirmou. No grupo não tem crianças nem idosos. Em comum os dados tem o gênero masculino e ausência de acompanhantes. Acredito que seja mais uma provável falha nos dados e para este caso vou atribuir o valor médio da coluna Fare.

In [ ]:
# Retorna a média da coluna Fare quando o valor passado como parametro é igual a 0.
titanic_df.Fare = titanic_df.Fare.apply(lambda x: titanic_df.Fare.mean() if x==0 else x)  

In [ ]:
# Sumariza os dados novamente
titanic_df.Fare.describe()

Como pode ser constatado, o valor mínimo (min) passou a ser 4.01 e o máximo(max) 263.

## Mais um pouco de estatítica
### Correlação de Pearson
Qual a corelação das variáveis como fator sobrevivência? O método pearson (.corr) foi utilizado para demonstrar a dependência  entre elas. O valor pode variar entre 0 e 1, tanto negativo quanto positivo.
Espera-se um grau de correlação superior a 0.5 para demonstrar uma correlação de moderada a muito forte.
    

In [ ]:
titanic_df.corr(method='pearson', min_periods=1)

Como a coluna Sex não é numérica, não foi calculada a correlação para ela. Acredito que ela tenha uma correlação com survived, então vou transformá-la em número, numa coluna adicional, para mensurar a correlação.

In [ ]:
# Atribuí um valor inteiro para os valores categorizados de Sex (0 - female, 1 - male)
titanic_df['SexInt'] = map(int, titanic_df.Sex == 'male')

In [ ]:
# Chama o método de corrlação passando o tipo pearson como parâmetro e guarda os valores em um dataframe.
correlation_df = titanic_df.corr(method='pearson', min_periods=1).abs()
correlation_df

O objetivo desta análise é pontuar a correlação entre as variáveis do nosso conjunto de dados, sem intenção nenhuma de estabelecer, através destes números, uma causa para a sobrevivência ou definir valores estatísticos sem teste controlado associado.

Foi importante adicionar a SexInt para representar a variável Sex. As variáveis Sex(SexInt) e Pclass são as mais correlacionadas com Survived. Faz sentido se pensarmos que a localização da cabine pode ter feito diferença e que as mulheres e crianças tiveram prioridade numa situação de emergência.
Pclass e Fare também estão correlacionadas, principalmente porque o tipo/classe cabine são fatores para composição dos valores de passagens.

**Classificação do resultado - correlação _p_**

In [ ]:
correlation_df.unstack().sort_values(ascending=False)

=> Correlação moderada _p_>0.5
* Survived x SexInt
* Pclass   x Fare

=> Correlação fraca _p_ >0.3
* Pclass   x Survived
* Pclass   x Age

=> Correlação desprezível
* Survived x Fare    
* Family   x Fare
* Family   x Age
* Fare     x SexInt 
* Family   x SexInt 
* Pclass   x SexInt     
* Age      x Fare       
* Age      x SexInt  
* Survived x Age 
* Pclass   x Family 
* Survived x Family 

## Visualizando os dados - Análise exploratória


Os dados estão limpos e prontos para serem visualizados. 

Começaremos a análise exploratória com uma visão geral de todas as colunas e o relacionamento dos seus dados. 
Os dados estão agrupados em 0 -mortos e 1 - sobreviventes  

In [ ]:
# Visão geral de todas as variáveis com a ajuda do pairplot(). 
sb.pairplot(titanic_df, hue='Survived', diag_kind='kde', size=2.5, markers=['o','s'], palette=['gray','red'])

Esta visualização geral auxilia na escolha de quais dados devem ser selecionados para uma exibição mais detalhada. Por exemplo, em SexInt está claro que mais homens morreram no Titanic. Em Pclass pode ser percebido um maior volume de mortos na 3º classe. 

### Visualizando Age e Pclass

Através dos gráficos tentaremos compreender melhor o conteúdo e importância da idade para a sobrevivência, além de seu relacionamento com a classe - Pclass.

In [ ]:
fig = plt.figure(figsize=(18,6), dpi=1600) 

# criando o subplot1 
ax1 = plt.subplot(2,2,1)
#Histogram da coluna Age
titanic_df.Age.hist(bins=10) 
#Definindo o label de x - Age
plt.xlabel("Age")
#Definindo o titulo do grafico
plt.title("Histrogram Age, (bin=10)")    

# criando o subplot2 
ax2 = plt.subplot(2,2,2)
#Plotando o grafico de densidade da coluna Age
titanic_df['Age'].plot(kind='kde', style='k--')
#Definindo o label do eixo y
plt.ylabel("Density")
#Definindo o label do eixo de x
plt.xlabel("Age")
#Definindo o titulo do grafico
plt.title("Densidade - Age")

# criando o subplot3
ax3 = plt.subplot(2,2,(3,4))
#plotando a densidade por classe
titanic_df.groupby('Pclass').Age.plot.kde()
#Definindo o label do eixo de x
plt.xlabel("Age")  
#Definindo o titulo do grafico
plt.title("Distribuicao Age/Class")
#Definindo a legenda
plt.legend(('1 Classe', '2 Classe','3 Classe'),loc='best') 

- A maior parte dos passageiros tem entre 20 e 40 anos.
- A 3º classe tem a maioria dos passageiros jovens e a 1º tem uma distribuição mais uniforme das idades, inclusive com a maior parte dos passageiros idosos.
- A distribuição do histograma da idade é normal.

### Visualizando Survived, Pclass e Sex

In [ ]:
fig = plt.figure(figsize=(18,6), dpi=1600) 
# tentar subplot com sb
# Cria o subplot1 
ax1 = plt.subplot(1,2,1)
# Imprime gráfico com a densidade de sobreviventes por classe
titanic_df.groupby('Pclass').Survived.plot.kde()

# Define o label dos eixos x e y
plt.xlabel("0 - Mortos  1 - Sobreviventes")
plt.ylabel("Densidade")
# Define o título do gráfico
plt.title("Distribuicao Sobreviventes por Classe")
# Define a legenda
plt.legend(('1 Classe', '2 Classe','3 Classe'),loc='best') 

# Cria o subplot2 
ax2 = plt.subplot(1,2,2)
# Imprime gráfico com o total de homens e mulheres no titanic
titanic_df.groupby('Sex').count()['Survived'].plot.bar()
plt.xlabel("Feminino - Masculino")
# Define o título do gráfico
plt.title("Total Homens e Mulheres")

* A distribuição dos mortos e sobreviventes por classe não deixa dúvida da relação entre essas variáveis. O maior volume de mortos estão na 3º classe e de sobreviventes na 1º. A segunda classe tem uma pequena variação. A relação entre a classe e a sobrevivência foi uma das questões levantandas no início do do trabalho. Não é possível afirmar que estar em uma determinada classe seja a causa da sobrevivência, através dos dados só é possível observar que a maior parte dos sobreviventes estava na 1º classe.
* No conjunto de dados a maioria dos passageiros eram homens.

In [ ]:
# Cria série com os dados de todos os sobreviventes
sobreviventes = titanic_df[titanic_df['Survived']==1]

# Para os sobreviventes, calcula a média da idade por classe e gênero  
#sobreviventes.groupby(['Pclass','Sex']).mean()['Age'].plot.bar()
sb.factorplot(x="Sex", y="Age", hue="Pclass",
               col="Pclass", data=sobreviventes, kind="box", size=4, aspect=.5)

# Define configurações do grafico 
plt.xlabel("Genero")
plt.ylabel('Idade')

* Através dos gráficos acima começamos a traçar parte do perfil dos sobreviventes, sabemos que a maioria estava na terceira classe e que tinham entre 20 e 40 anos.
* Na 2º classe, objeto da investigação, a maioria das sobreviventes tinha idade entre 25 e 35 anos.
* A maior parte dos homens sobreviventes da 2º classe tinham no máximo 30 anos.

### Respondendo aos questionamentos iniciais

Agora que conhecemos melhor os dados e já começamos a identificar as características mais comuns aos sobreviventes, tentaremos responder aos questionamentos que motivaram esta análise com uma investigação mais detalhada.

** Questão 1: Sabendo que houve esse mal entendido na distribuição dos passageiros por botes, a fonte de dados titanic.cvs nos fornece alguma informação que reflete esse mal entendido entre os oficiais? **

Sabemos até o momento que a maioria dos passageiros eram homens, Jovem adulto era a faixa estária mais comum e que a maior parte dos mortos estava na 3º classe. Os oficiais do Titanic priorizaram crianças e mulheres e é de se esperar, por esta razão, que os homens tenham uma taxa de sobrevivência menor. 

Não será feita nenhuma análise detalhada na coluna Age, usaremos a coluna derivada AgeRange que contém a faixa etária do passageiro.

In [ ]:
# Imprime violinplot com a distribuição dos sobreviventes por sexo e faixa etária
ax = sb.violinplot(data=titanic_df, x='SexInt', y='Survived', hue='AgeRange')
ax.set(xlabel='(0)Mulheres , (1)Homens', ylabel='(0)Mortos, (1)Sobreviventes')

No gráfico acima é bem nítido que, pelo menos na amostra que nos foi dada, a maioria das mulheres sobreviveu independente da faixa etária. A maior parte dos homens morreu, inclusive os idosos. Tanto no grupo de homens e mulheres o volume de crianças sobreviventes é similar ao de mortos. Médias para o grupo Faixa Etária e Gênero.

** Questão 2: Mulheres e crianças tiveram maior chance de sobreviver? **

Vamos começar com a taxa de sobrevivência de homens e mulheres independente da idade.

In [ ]:
# Agrupa os dados por gênero
titanic_df.groupby(['Sex']).mean()

No conjunto de dados 74% das mulheres e 19% dos homens sobreviveram. Boa notícia para mim, mas será que se eu adicionar a Faixa Etária, o percentual de sobrevivência continua favorável? 

In [ ]:
# Agrupa os dados em faixa etária e gênero 
titanic_df.groupby(['AgeRange','Sex']).mean()

No grupo de jovem adultas mulheres a taxa de sobrevivência é de 71%, bem superior aos 16% dos homens nesta faixa etária. Com exceção das crianças, os homens em todas as faixas etárias tiveram um taixa baixa de sobrevivência.
As crianças, independente do sexo, tiveram uma taxa similiar, sendo 59% para as meninas e 56% para os meninos. Reflete bem a informação exibida pelo violinplot, nele conseguimos perceber uma distribuição bem igualitária na distribuição de crianças mortas e sobreviventes.

Aprofundando na análise e adicionando a faixa etária a chance de sobrevivência do meu grupo foi reduzida em 3%, antes era 74% e agora passou para 71%. Entre os valores apurados para as mulheres, este é um dos mais baixos, perde apenas para as crianças, mas ainda estou feliz porque falta analisar mais um elemento: a classe.

** Questão 3: A classe, foi um fator importante para a sobrevivência? **

Para descobrir se era mais provável sobreviver em determinadas classes do que outras, vamos investigar a taxa de sobrevivência somente da variável classe, sem interferência de outra como Sex ou AgeRange.

In [ ]:
# Primeiro vamos agrupar os dados somente pela classe
titanic_df.groupby(['Pclass']).mean()

Este agrupamento confirma o que vimos nos gráficos, o índice de sobrevivência é significativamente menor para 3º classe.
Esta informação sozinha não significa muita coisa, porque sabemos que mulheres e crianças tinham melhores chances. Vejamos a distribuição dos valores com a adição de Sex e AgeRange.

In [ ]:
# Agrupa os dados por Faixa etária, Gênero, Classe
faixa_etaria_genero = titanic_df.groupby(['AgeRange','Sex','Pclass']).mean()
faixa_etaria_genero

De acordo com a nossa amostra, pobre dos homens adultos e idosos na 3º classe, a taxa de sobrevivência deles é inferior a 4%.

** 4º questão: E a pergunta me faço é se eu estivesse lá, mulher, jovem adulta, na 2º classe é provável que teria sobrevivido? **

O que sabemos até agora:
- Mulheres tiveram taxas de sobrevivência maiores quando comparadas a de homens
- Crianças tiveram taxas de sobrevivência maiores que os homens
- A maioria dos passageiros mortos estava na 3º classe
- A taxa de sobrevivência dos passageiros da 1º classe foi superior do que a da 2º classe

Para responder a esta qustão vamos pinsar o grupo específico: 

In [ ]:
jovem_adulta = titanic_df.groupby(['AgeRange','Sex','Pclass']).mean().T
jovem_adulta

### <font color='Red'>Muito bom! No conjunto de dados 92% das jovens adultas na 2º classe sobreviveram.</font> 
Os dados que temos são somente uma amostra dos passageiros do Titanic, alguns dados estavam ausentes, outros foram excluídos e uma boa parte das informações foi preenchida com valores supostos, por este motivo, a taxa aqui apresentada só é valida como experiência neste contexto, não significando que, uma Jovem adulta na 2º classe teria sobrevivido.

## Prevendo o resultado

A partir de agora vamos deixar nossa análise mais interessante, vamos adicionar um pouco de machine learning ao treinar um programa na amostra de dados do Titanic. Depois de "treinado" testaremos a predição para uma Jovem Adulta na 2º classe. Será que obteremos um resultado diferente da análise exploratória?

** Preparando os dados**
- Na fase de análise de dados eu criei a coluna SexInt, que nada mais é do que a coluna Sex em uma versão numérica: 0-Female, 1-Male, agora vou excluir essa coluna e usar o LabelEnconder para fazer a mudança dos valores.
- A coluna AgeRange também será submetida ao LabelEnconder, porque durante a frase exploratória através desta variável pudemos ver informações que acrescentaram valor a análise.

In [ ]:
titanic_df.head()

In [ ]:
# Cria uma cópia do titanic_df
processed_df = titanic_df.copy()
# Apaga a coluna SexInt na cópia do titanic_df
processed_df.drop('SexInt',axis=1, inplace=True)
processed_df.head()

Os dados nas colunas AgeRange e Sex são categorizados e serão transformados com a ajuda do LabelEncoder em valores numéricos que são lidos mais facilmente pelo modelo.

In [ ]:
le = preprocessing.LabelEncoder()

# As colunas Sex e AgeRange recebem sua versão numerica
processed_df.Sex = le.fit_transform(processed_df.Sex)
processed_df.AgeRange = le.fit_transform(processed_df.AgeRange)

In [ ]:
# Conferindo o resultado
processed_df.head()

In [ ]:
# X recebe todos os valores do dataset menos a coluna Survived que será usada na comparação y
X = processed_df.drop(['Survived'], axis=1).values

# y recebe os valores da coluna Survived que será usada pelo modelo como comparação
y = processed_df['Survived'].values

Depois de criado os dois conjuntos de dados, X contendo todos os valores menos Survived e y que contém só os valores de Survived (resultado esperado), os dados serão divididos nos dataset's de treino e teste. O modelo será treinado pelo algoritmo de classificação usando o X_train e y_train. Depois que o modelo estiver pronto, eles serão usados para classificar o teste.

In [ ]:
# Divide as matrizes em teste e treino
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2)

Treinando o modelo:

In [ ]:
# Cria o objeto Decision Tree
clf_dt = tree.DecisionTreeClassifier(max_depth=5)

# Treinando o modelo
clf_dt.fit(X_train, y_train)

** Acurácia média dos dados de teste **

In [ ]:
# Verifica a precisao do modelo.
clf_dt.score (X_test, y_test)

O modelo da árvore de decisão conseguiu prever a sobrevivência de no mínimo 82% dos dados.

** Dados simulados **

Com base nas informações que já verificamos até aqui, vamos testar o modelo inserindo dados simulados da passageira principal (Jovem adulta, 2º classe), faremos a variação da classe para esta mesma passageira e incluiremos mais dois passageiros que estão em um grupo com baixa taxa de sobrevivência. Será que o resultado será similar ao da análise exploratória?

Conjunto que testaremos:

| Pclass |  Sex    | Age | Fare | AgeRange   |
|--------|---------|-----|------|------------|
|  1     |Feminino | 32  | 109  |Jovem Adulto|
|  2     |Feminino | 32  | 21   |Jovem Adulto|
|  3     |Feminino | 32  | 14   |Jovem Adulto|
|  3     |Masculino| 80  | 23   |Idoso       |
|  2     |Masculino| 32  | 8    |Jovem Adulto|

In [ ]:
# Cria dataset com as informações dos passageiros simulados
# Features: Pclass, Sex, Age, Fare, Family, AgeRange
passageiros_simulados = [[1, 0, 32, 109, 0, 4],
                         [2, 0, 32, 21, 0, 4],
                         [3, 0, 32, 14, 0, 4],
                         [3, 1, 80, 23, 5, 3],
                         [2, 1, 32, 8, 0, 4]]

# Predição para o conjunto de dados simulados
clf_dt.predict(passageiros_simulados)

Relembrando: 0 - Morreu, 1 - Sobreviveu

Que interessante! O resultado obtido com o modelo Decision Tree prevê que uma jovem adulta, na 1º ou 2º classe sobreviveria.
Já para a 3º classe não foi predito sobrevivência.
Para testar a "competência" do modelo em prever resultados, mudei o sexo da passageira principal, porque na fase de análise os homens tiveram uma taxa de sobrevivência menor, e como era de se esperar, o modelo previu a morte para ele. Cruel, mas dentro da expectativa. Sem surpresas nenhuma também, foi prevista a morte para nosso passageiro de 80 anos.

Gerando o arquivo.dot com as decisões do modelo Decision Tree:

In [ ]:
#Gera a representação GraphViz da decision tree. O dado é gravado no arquivo titanic_tree.dot
#Os dados podem ser visualizados graficamente em http://www.webgraphviz.com/
tree.export_graphviz(clf_dt, out_file='titanic_tree.dot', feature_names=processed_df.columns[1:])

O arquivo 'titanic_tree.dot' está no github: https://github.com/liebycardoso/Intro_Data_Analysis

A fim de facilitar a visualização, foi gerado a representação GraphViz da decision tree com max_depth=5.

![alt text](Tree_depth5.jpg "tree")

- A árvore começa decidindo entre homens e mulheres (Sex <= 0.5).
- O ramo que nos interessa é o True porque as mulheres são representadas pelo valor 0.
- Na sequência ele avalia a classe (Pclass <= 2.5), o que inclui a 2º classe.
- Em outro nó ele seleciona Fare <= 22.90, o que estamos pesquisando também está incluído nessa amostra porque o preço médio da passagem de uma jovem mulher na segunda classe é 21.15.
- Também é avaliado Age <= 36.5 e em outro ramo segue avaliando Family <= 1.5
- Para esse grupo também avalia Fare e Age. 
- O coeficiente de Gini nas decisões atinge no máximo 0.5 demonstrando uma tendência a igualdade da amostra.

** Utilizando o modelo Random Forest **

 O decisionTree é um modelo simples de árvore de decisão, vou treinar os dados no random forest também porque ele vai decidir utilizando múltiplas árvores e retornar o resultado mais comum entre elas.

In [ ]:
# Cria o objeto Random Forest
clf_rf = RandomForestClassifier(n_estimators=100, oob_score=True)

# Treina o modelo de dados da mesma forma que foi feito para o DecisionTreeClassifier 
clf_rf.fit(X_train, y_train)

** Prevendo o resultado **

In [ ]:
# Prediz o resultado
Y_pred = clf_rf.predict(X_test)

In [ ]:
# Verifica a precisão do modelo.
clf_rf.score(X_train, y_train)

Neste caso é possível prever a sobrevivência com 98% de precisão.

In [ ]:
# Estimatica do out-of-bag (oob) error: 81%
clf_rf.oob_score_

###  Usando os dados simulados para prever o resultado com o Random Forest

Vamor testar o mesmo conjunto de dados usado com o Decision Tree para prever o resultado com o Random Forest. Dessa forma conseguiremos comparar o resultado dos dois.

In [ ]:
# Predição para o conjunto de dados simulados
clf_rf.predict(passageiros_simulados)

** Comparativo entre o resultado do Decision Tree e Random Forest: **

| Pclass |  Sex    | Age | Fare | AgeRange   | Decision Tree | Random Forest|
|--------|---------|-----|------|------------|---------------|--------------|
|  1     |Feminino | 32  | 20   |Jovem Adulto|  Sobreviveu   | Sobreviveu   |
|  2     |Feminino | 32  | 20   |Jovem Adulto|  Sobreviveu   | Sobreviveu   |
|  3     |Feminino | 32  | 20   |Jovem Adulto|  Sobreviveu   | Morreu   |
|  3     |Masculino| 80  | 20   |Idoso       |  Morreu       | Morreu       |
|  2     |Masculino| 32  | 20   |Jovem Adulto|  Morreu       | Morreu       |

Os dois modelos retornaram um resultado diferente apenas para um registro do conjunto de dados. Para este registro é compreensível a diferença, uma vez que, durante a análise exploratória vimos que na 3º classe a taxa de sobreviência para a jovem adulta foi 52%. Então, morte ou sobrevivência são dois resultados possíveis para esta passageira. A taxa da 1º classe foi 97% e da 2º classe foi 92%. E o mais interessante é que este resultado está em consonância com as conclusões que tivemos durante a análise dos dados.
Mulher, Jovem Adulta, tanto na 1º ou na 2º classe apresentaram uma boa probabilidade de sobrevivência nos modelos e na análise exploratória. 
Para os homens o resultado também está de acordo com a análise, já que a todo tempo a taxa de sobrevência para eles mostrou-se baixa.

** A importância de cada coluna na obtenção do resultado **

Abaixo será listado por ordem de importância de cada variável (feature) usada no modelo para o resultado final.

In [ ]:
feat_importance = pd.Series(clf_rf.feature_importances_, index=processed_df.drop(['Survived'], axis=1).columns)
feat_importance.sort_values(ascending=False)

Gráfico com a importância de cada feature:

In [ ]:
plt.barh(np.arange(len(feat_importance)), feat_importance, alpha=0.7)
plt.yticks(np.arange(.5,len(feat_importance),1), feat_importance.index)
plt.xlabel('Importancia')
plt.ylabel('Variavel')
plt.title('Importancia de cada variavel')

A hierarquia de importância de cada variável é bem próxima da obtida na correlação de pearson. Sendo Fare, Sex e Age as mais correlacionadas com os dados.

## Comparando o resultado - Titanic_df vs Y_pred

Será criado um dataframe com a predição (Y_pred) e os valores do X_test. É um dataframe com o objetivo de simular o resultado do conjuto de dados do Titanic.

In [ ]:
# Cria o dataframe
predicao_df = pd.DataFrame(X_test, columns=['Pclass','Sex', 'Age', 'Fare', 'Family','AgeRange'])
predicao_df['Predict'] = Y_pred
predicao_df['Survived'] = y_test

In [ ]:
predicao_df.head()

Verificando os valores para a nova coluna Predict:

In [ ]:
predicao_df.groupby(['Predict']).mean()

O mesmo agrupamento, só que utilizando o titanic_df como dataset:

In [ ]:
titanic_df.groupby(['Survived']).mean()

Como pode ser percebido, os valores são bem aproximados. Por exemplo:
- Para os sobreviventes a idade média era 29 e no modelo predito foi 32;
- Para os sobreviventes o valor médio da passagem era 46 e no modelo predito foi 48;

Para facilitar a análise, vou plotar novamente Sex/Age/Pclass para os passageiros sobreviventes.
Primeiro será impresso os dados dos passageiros do titanic_df e na sequência do predicao_df.

In [ ]:
# Imprime gráfico com o bloxpot da idade média por genero e classe no dataframe original titanic_df
sb.factorplot(x="Sex", y="Age", hue="Pclass",
               col="Pclass", data=sobreviventes, kind="box", size=4, aspect=.5)

plt.title("Dados titanic_df")  

# Imprime gráfico com o bloxpot da idade média por genero e classe no dataframe predicao_df
predicao_sobrevivente = predicao_df[predicao_df['Predict']==1]
sb.factorplot(x="Sex", y="Age", hue="Pclass",
               col="Pclass", data=predicao_df, kind="box", size=4, aspect=.5)
plt.title("Dados predicao_df")  

In [ ]:
predicao_sobrevivente[[]]

In [ ]:
# Média para o grupo
predicao_df.groupby(['Sex','Pclass','Predict']).mean()['Age']

In [ ]:
# Total de sobreviventes por Sex/Pclass
predicao_df.groupby(['Sex','Pclass','Predict']).count()['Age'].unstack()

Analisando a tabela acima conseguimos perceber que o modelo previu a sobrevivência para todas as passageiras da 1º e 2º Classes.

Filtros:
- Sex = 0
- Pclass = 1 e 2
- Predict = 0(Mortos): Nenhum valor - NaN
- Predict = 1(Sobreviventes): 24 + 17

## Conclusão

Após investigar o conjunto de dados do Titanic verificamos que os as mulheres tiveram um percentual de sobrevivência superior ao dos homens, que os passageiros da 1º e 2º classe também tiveram taxas melhores. Neste cenário foi investigado se uma Jovem adulta na 2º classe sobreviveria, e assuminos, baseado nos dados e na predição dos modelos Decision Tree e Random Forest, que sim, essa passageira teria uma boa chance de sobreviver.

## Limitações da análise

As observações, constatações e resultados não representam a realidade dos fatos, porque  estamos trabalhando somente com uma amostra incompleta dos dados.
 
A amostra tem informações sobre 891 passageiros e é sabido que 2.223 pessoas estavam a bordo do Titanic. A soma de mortos é superior ao tamanho da amostra:
- 832 – passageiros mortos
- 685 – membros da tripulação mortos
- 706 – sobreviventes, entre passageiros e tripulação
 
Uma outra limitação da análise são os valores que foram assumidos no lugar dos valores ausentes que possivelmente afetaram o resultado final. Algumas informações que foram criadas para auxiliar a análise:
 
1)Idade: 
Os valores nulos foram substituidos pela média geral da coluna Age.

2)Preço da passagem: 
O maior valor da passagem estava muito distante do segundo maior valor, então substituí o maior valor pelo segundo maior.
Os valores zerados foram substituídos pela média geral da coluna Fare. 

A coluna Cabin foi excluída da análise por ter muitos valores nulos, mas os valores desta variável sugerem um padrão que pode ser investigado numa nova análise. O mesmo acontece com a informação do tratamento dado ao passageiro (Miss, Mrs, Mr, Master, Major e etc.), será que conseguimos estabelecer um padrão também para esta coluna? E este tratamento ajudaria na previsão dos resultados? Estas são algumas perguntas que não foram respondidas aqui e que por terem sido ocultadas, podem ter interferido no resultado final.

Apesar do modelo supervisionado Random Forest ter-se aplicado bem a estes dados, outro modelo, como support vector machine (SVM) pode ser testado, caso você tenha interesse em ampliar esta investigação.

Um  fator muito importante é que os dados referem-se a seres humanos e seu comportamento e decisões no momento de risco contam com muitas variáveis que são desconhecidas por este análise. Por exemplo, os sobreviventes relataram a dificuldade em convencer alguns passageiros a embarcar, imagino que algumas mulheres tiveram dificuldades de deixar seus maridos e filhos mais velhos, talvez elas nem tenham deixado. O entendimento incorreto das ordem do capitão fez com que alguns homens em determinada localização do navio fossem autorizados a embarcar e os homens do outro lado do navio não receberam essa autorização e acabaram contribuindo para o redução da taxa de sobrevivência do gênero Masculino. 

Por existirem tantas variáveis e a falta de algumas informações não podemos atribuir um valor estatítico a este trabalho.

https://pt.wikipedia.org/wiki/RMS_Titanic


### BIBLIOGRAFIA

- Wes McKinney; Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython

Sites consultados:
- http://blog.socialcops.com/engineering/machine-learning-python
- http://developers.hekima.com/machine%20learning/python/2016/05/17/churn-prediction/
- http://hamelg.blogspot.com.br/2015/11/python-for-data-analysis-part-14.html
- http://nbviewer.jupyter.org/github/agconti/US_Dollar_Vehicle_Currency/blob/master/US_Dollar_Vehicle_Currency.ipynb
- http://nbviewer.jupyter.org/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-4-Matplotlib.ipynb
- http://scott.fortmann-roe.com/docs/BiasVariance.html
- http://stackoverflow.com/questions/36288352/how-to-get-pandas-kde-density-with-horizontal-orientation
- http://work.caltech.edu/library/014.html
- http://work.caltech.edu/library/index.html
- http://www.r2d3.us/visual-intro-to-machine-learning-part-1/
- http://www.reshamsarkar.com/projects/2016/3/30/titanic-trauma-data-cleaning-and-munging
- http://www.ultravioletanalytics.com/2014/11/03/kaggle-titanic-competition-part-ii-missing-values/
- https://github.com/agconti/kaggle-titanic/blob/master/Titanic.ipynb
- https://github.com/justmarkham/scikit-learn-videos
- https://www.dataquest.io/mission/3/list-operations
- https://www.kaggle.com/c/titanic
- https://www.youtube.com/watch?v=0GrciaGYzV0
- http://www.sbmac.org.br/cmacs/cmac-ne/2012/trabalhos/PDF/122.pdf
- http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html